# 📘 Feature Engineering & Encoding Notebook

### **🎯 Objective**

Convert the cleaned dataset into a model-ready feature matrix by encoding categorical variables and assembling numeric features without re-cleaning, scaling, or leakage. This notebook produces a stable features artifact for modeling.

### Executive Summary

###  Notebook

**File:** `notebooks/03_feature_engineering.ipynb`

**Stage:** Feature Engineering & Encoding

**Input Contract:** `data/processed/nwmp_cleaned_v1.csv`

**Output Contract:** `data/processed/nwmp_features_v1.(csv | parquet)`

---

### Objective

Transform the **cleaned and validated dataset** into a **model-ready feature matrix** by:

* encoding categorical variables,
* assembling numeric and engineered features,
* enforcing strict schema invariants,

**without performing cleaning, imputation, scaling, or modeling decisions.**

This notebook establishes a **stable, reusable feature representation** for all downstream models.

---

### Input Assumptions (Contract Enforcement)

The input dataset satisfies the following invariants:

* ✅ No missing values (NaNs)
* ✅ Leakage-prone, metadata, and sparse columns already removed
* ✅ Numeric features properly typed
* ✅ BDL (Below Detection Limit) information preserved as binary flags
* ✅ One row = one observation
* ✅ Target variable (`use_based_class`) present and clean

If any of these conditions fail, the pipeline must return to **data cleaning**.

---

### Feature Engineering Scope

#### Included Operations

* Target separation (`X`, `y`)
* Identification of categorical features
* One-hot encoding of low-cardinality categorical columns
* Assembly of numeric + encoded categorical features
* Schema validation (row alignment, NaN checks)
* Export of model-ready feature matrix

#### Explicitly Excluded Operations

* ❌ Data cleaning or imputation
* ❌ Column dropping or leakage decisions
* ❌ Feature scaling or normalization
* ❌ Outlier handling
* ❌ Feature selection
* ❌ Model training

---

### Feature Composition

#### Numeric Features

* Physicochemical parameters (pH, DO, conductivity, TDS, turbidity, etc.)
* Chemical contaminants (nutrients, ions, hardness, alkalinity)
* Biological indicators (fecal coliform, total coliform, streptococci)
* Engineered **BDL indicator flags** (`*_is_bdl`)

All numeric features are passed through **unchanged**.

---

#### Categorical Features

* Domain-relevant, low-cardinality features only
* Encoded using **one-hot encoding**
* No identifiers, metadata, or leakage-prone columns included

High-cardinality or free-text columns are intentionally excluded upstream.

---

### Validation & Safety Checks

Before export, the following conditions are enforced:

* Feature matrix contains **no missing values**
* Feature rows align exactly with target labels
* No duplicate rows introduced during encoding

These checks ensure downstream models receive a **stable and deterministic input**.

---

### Output Artifact

The notebook exports a single, versioned feature dataset:

* **CSV (mandatory):** transparent, debuggable
* **Parquet (optional):** optimized for performance and scale

This artifact represents the **final feature contract** for all modeling notebooks.

---

### Key Design Principles

* **Separation of concerns:**
  Cleaning → Feature Engineering → Modeling are strictly isolated.

* **Reproducibility:**
  Feature generation is deterministic and independent of model choice.

* **Leakage safety:**
  Only features that exist prior to labeling are included.

* **Scalability:**
  Same features can be reused across multiple models and experiments.

---

### Status

✅ Feature engineering complete
✅ Schema validated
✅ Model-ready dataset exported
🟢 **Ready for model training**



### **Imports**

In [1]:
import pandas as pd
import numpy as np

import os
from pathlib import Path

In [2]:
from utils.config import DATA_DIR # Path to raw data source
from src.data_preprocessing.create_dataframe import create_dataframe

### **💿 Data Loading**

In [3]:
# ============================================================================
# DATA LOADING — CLEANED CONTRACT INPUT
# ============================================================================

INPUT_CSV = os.path.join(DATA_DIR, "processed", "csv", "cleaned_water_quality_data.csv")
# Optional parquet if available:
# INPUT_PARQUET = os.path.join(DATA_DIR, "processed", "parquet", "cleaned_water_quality_data.parquet")

df = create_dataframe(INPUT_CSV)

print("✓ Loaded cleaned dataset")
print(df.shape)


✓ Loaded cleaned dataset
(171, 56)


### Input Contract Validation

In [4]:
# ============================================================================
# INPUT CONTRACT VALIDATION
# ============================================================================
print("\n🔍 VALIDATING INPUT CONTRACT...")
print("="*80)

validation_passed = True

# Check 1: No missing values
missing_count = df.isna().sum().sum()
if missing_count == 0:
    print("  ✓ No missing values")
else:
    print(f"  ❌ FAILED: {missing_count} missing values found")
    validation_passed = False

# Check 2: Target exists and is clean
TARGET_COL = "use_based_class"
if TARGET_COL in df.columns:
    print(f"  ✓ Target column '{TARGET_COL}' present")
    
    target_missing = df[TARGET_COL].isna().sum()
    if target_missing == 0:
        print(f"  ✓ Target has no missing values")
    else:
        print(f"  ❌ FAILED: Target has {target_missing} missing values")
        validation_passed = False
else:
    print(f"  ❌ FAILED: Target column '{TARGET_COL}' not found")
    validation_passed = False

# Check 3: Verify expected classes
expected_classes = {'A', 'B', 'C', 'E'}
actual_classes = set(df[TARGET_COL].unique())

if actual_classes == expected_classes:
    print(f"  ✓ Target classes validated: {sorted(actual_classes)}")
elif actual_classes.issubset(expected_classes):
    print(f"  ⚠️ WARNING: Only {len(actual_classes)} classes present: {sorted(actual_classes)}")
    print(f"    Missing: {sorted(expected_classes - actual_classes)}")
else:
    print(f"  ❌ FAILED: Unexpected classes found")
    print(f"    Expected: {sorted(expected_classes)}")
    print(f"    Found: {sorted(actual_classes)}")
    validation_passed = False

# Check 4: No duplicate rows
dup_count = df.duplicated().sum()
if dup_count == 0:
    print("  ✓ No duplicate rows")
else:
    print(f"  ❌ FAILED: {dup_count} duplicate rows found")
    validation_passed = False

# Check 5: Reasonable shape
min_records = 50
min_features = 10

if df.shape[0] >= min_records:
    print(f"  ✓ Sufficient records: {df.shape[0]} (minimum: {min_records})")
else:
    print(f"  ❌ FAILED: Only {df.shape[0]} records (minimum: {min_records})")
    validation_passed = False

if df.shape[1] >= min_features:
    print(f"  ✓ Sufficient features: {df.shape[1]} (minimum: {min_features})")
else:
    print(f"  ❌ FAILED: Only {df.shape[1]} features (minimum: {min_features})")
    validation_passed = False

# Check 6: Data types are appropriate
numeric_cols = df.select_dtypes(include=['number']).columns
if len(numeric_cols) > 0:
    print(f"  ✓ Numeric features present: {len(numeric_cols)}")
else:
    print(f"  ⚠️ WARNING: No numeric features found")

print("="*80)

if validation_passed:
    print("\n✅ ALL INPUT CONTRACT VALIDATIONS PASSED!")
    print("   Dataset is ready for feature engineering\n")
else:
    print("\n❌ INPUT CONTRACT VALIDATION FAILED!")
    print("   Please return to data cleaning notebook to fix issues\n")
    raise ValueError("Input contract validation failed - cannot proceed with feature engineering")


🔍 VALIDATING INPUT CONTRACT...
  ✓ No missing values
  ✓ Target column 'use_based_class' present
  ✓ Target has no missing values
  ✓ Target classes validated: ['A', 'B', 'C', 'E']
  ✓ No duplicate rows
  ✓ Sufficient records: 171 (minimum: 50)
  ✓ Sufficient features: 56 (minimum: 10)
  ✓ Numeric features present: 29

✅ ALL INPUT CONTRACT VALIDATIONS PASSED!
   Dataset is ready for feature engineering



### **Define Target & Separate Features**

In [5]:
# ============================================================================
# TARGET SEPARATION
# ============================================================================

TARGET_COL = "use_based_class"

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].copy()

print("✓ Features / target separated")
print("X shape:", X.shape, "| y shape:", y.shape)


✓ Features / target separated
X shape: (171, 55) | y shape: (171,)


### Feature Engineering (Encoding Scope Only)


**3.1 Identify categorical columns to encode**

(Low-cardinality only; no identifiers, no leakage, no free-text)

In [6]:
# ============================================================================
# CATEGORICAL FEATURE SELECTION (ENCODING SCOPE)
# ============================================================================

categorical_cols = X.select_dtypes(include=["object", "string"]).columns.tolist()

# Explicit exclusions (already decided in EDA / cleaning)
EXCLUDE_CATS = [
    # identifiers / metadata / free-text already removed upstream
]

# Keep only safe, low-cardinality categories
encode_cats = [c for c in categorical_cols if c not in EXCLUDE_CATS]

print("Categorical columns to encode:", encode_cats)


Categorical columns to encode: ['type_water_body', 'river_basin', 'district', 'weather', 'approx_depth', 'human_activities', 'floating_matter', 'color', 'odor']


### Categorical Feature Analysis

**Objective:** Understand categorical features before encoding

In [7]:
# ============================================================================
# CATEGORICAL FEATURE PRE-ENCODING ANALYSIS
# ============================================================================
print("\n📊 CATEGORICAL FEATURES PRE-ENCODING ANALYSIS")
print("="*80)

if len(encode_cats) > 0:
    print(f"Found {len(encode_cats)} categorical features to encode:\n")
    
    for col in encode_cats:
        n_unique = X[col].nunique()
        n_missing = X[col].isna().sum()
        top_values = X[col].value_counts().head(3)
        
        print(f"\n{col}:")
        print(f"  Unique values: {n_unique}")
        
        if n_missing > 0:
            print(f"  ⚠️ Missing: {n_missing} ({n_missing/len(X)*100:.1f}%)")
        
        print(f"  Top 3 categories:")
        for val, count in top_values.items():
            pct = count / len(X) * 100
            print(f"    • {val}: {count} ({pct:.1f}%)")
        
        # Warning for high cardinality
        if n_unique > 20:
            print(f"  ⚠️ WARNING: High cardinality ({n_unique}) - will create {n_unique} dummy columns")
            print(f"     Consider target encoding or dropping this feature")
        elif n_unique > 10:
            print(f"  ℹ️ INFO: Medium cardinality ({n_unique}) - will create {n_unique} dummy columns")
    
    print("\n" + "="*80)
    
    # Summary statistics
    total_dummies = sum(X[col].nunique() for col in encode_cats)
    print(f"\n📊 Encoding Summary:")
    print(f"   Categorical features to encode: {len(encode_cats)}")
    print(f"   Total dummy columns to create: {total_dummies}")
    print(f"   Original feature count: {X.shape[1]}")
    print(f"   Expected feature count after encoding: {X.shape[1] - len(encode_cats) + total_dummies}")
    
    # Warning if dimension explosion
    if total_dummies > 50:
        print(f"\n   ⚠️ WARNING: Encoding will create {total_dummies} dummy columns!")
        print(f"      This may lead to:")
        print(f"      • High dimensionality (curse of dimensionality)")
        print(f"      • Increased memory usage")
        print(f"      • Longer training times")
        print(f"      Consider using alternative encoding methods")
    
    print("="*80)
else:
    print("No categorical features to encode")
    print("="*80)


📊 CATEGORICAL FEATURES PRE-ENCODING ANALYSIS
Found 9 categorical features to encode:


type_water_body:
  Unique values: 7
  Top 3 categories:
    • River: 133 (77.8%)
    • Creek: 18 (10.5%)
    • Nala: 9 (5.3%)

river_basin:
  Unique values: 65
  Top 3 categories:
    • Godavari: 31 (18.1%)
    • Godavri : 8 (4.7%)
    • Patalganga : 7 (4.1%)
  ⚠️ WARNING: High cardinality (65) - will create 65 dummy columns
     Consider target encoding or dropping this feature

district:
  Unique values: 20
  Top 3 categories:
    • Thane: 35 (20.5%)
    • Raigad: 20 (11.7%)
    • Nashik: 15 (8.8%)
  ℹ️ INFO: Medium cardinality (20) - will create 20 dummy columns

weather:
  Unique values: 3
  Top 3 categories:
    • Clear: 139 (81.3%)
    • Raining: 19 (11.1%)
    • Cloudy: 13 (7.6%)

approx_depth:
  Unique values: 3
  Top 3 categories:
    • Less than 50cm: 149 (87.1%)
    • Greater than 100cm: 12 (7.0%)
    • 50-100cm: 10 (5.8%)

human_activities:
  Unique values: 18
  Top 3 categories:
    • O

### One-hot encode categorical features

In [8]:
# ============================================================================
# ONE-HOT ENCODING (NO SCALING, NO LEAKAGE)
# ============================================================================

X_encoded = pd.get_dummies(
    X,
    columns=encode_cats,
    drop_first=False
)

print("✓ Encoding complete")
print("Encoded shape:", X_encoded.shape)


✓ Encoding complete
Encoded shape: (171, 179)


### ✅ Encoding Transformation Complete

**Objective:** Document the impact of one-hot encoding on feature space

In [9]:
# ============================================================================
# ENCODING IMPACT SUMMARY
# ============================================================================
print("\n📊 ENCODING TRANSFORMATION SUMMARY")
print("="*80)

print(f"Original Features:    {X.shape[1]}")
print(f"Encoded Features:     {X_encoded.shape[1]}")
print(f"Features Added:       {X_encoded.shape[1] - X.shape[1]}")
print(f"Categorical Encoded:  {len(encode_cats)}")

print("\n" + "="*80)

if len(encode_cats) > 0:
    print("\n📋 Categorical Feature Expansion:")
    print("-" * 80)
    
    for cat_col in encode_cats:
        # Find new columns created from this categorical
        new_cols = [col for col in X_encoded.columns if col.startswith(f"{cat_col}_")]
        
        print(f"\n{cat_col}:")
        print(f"  Original unique values: {X[cat_col].nunique()}")
        print(f"  Dummy columns created: {len(new_cols)}")
        
        # Show column names if not too many
        if len(new_cols) <= 10:
            print(f"  New columns:")
            for new_col in new_cols:
                # Extract the category value from column name
                category = new_col.replace(f"{cat_col}_", "")
                count = X[cat_col].value_counts().get(category, 0)
                print(f"    • {new_col} (n={count})")
        else:
            print(f"  ℹ️ {len(new_cols)} columns created (too many to display)")
            # Show just first 5
            print(f"  First 5 columns: {', '.join(new_cols[:5])}")
    
    print("\n" + "="*80)

print("\n✅ Encoding complete - feature matrix ready for modeling")


📊 ENCODING TRANSFORMATION SUMMARY
Original Features:    55
Encoded Features:     179
Features Added:       124
Categorical Encoded:  9


📋 Categorical Feature Expansion:
--------------------------------------------------------------------------------

type_water_body:
  Original unique values: 7
  Dummy columns created: 7
  New columns:
    • type_water_body_ Dam (n=1)
    • type_water_body_Creek (n=18)
    • type_water_body_Dam (n=3)
    • type_water_body_Lake (n=1)
    • type_water_body_Nala (n=9)
    • type_water_body_River (n=133)
    • type_water_body_Sea (n=6)

river_basin:
  Original unique values: 65
  Dummy columns created: 65
  ℹ️ 65 columns created (too many to display)
  First 5 columns: river_basin_Arnala , river_basin_BPT Navapur , river_basin_Bassein , river_basin_Bhagwati Bundar, river_basin_Bhatsa 

district:
  Original unique values: 20
  Dummy columns created: 20
  ℹ️ 20 columns created (too many to display)
  First 5 columns: district_Akola, district_Amravati, dist

In [10]:
# ============================================================================
# FINAL FEATURE COMPOSITION
# ============================================================================
print("\n📋 FINAL FEATURE MATRIX COMPOSITION")
print("="*80)

# Identify feature types in encoded data
numeric_features = X_encoded.select_dtypes(include=[np.number]).columns.tolist()

# Identify binary/boolean features (including one-hot encoded)
binary_features = []
for col in X_encoded.columns:
    if X_encoded[col].nunique() == 2:
        binary_features.append(col)

# One-hot encoded features
one_hot_features = [col for col in X_encoded.columns if col not in X.columns]

print(f"\nTotal Features:          {X_encoded.shape[1]}")
print(f"  Numeric (continuous):  {len([f for f in numeric_features if f not in binary_features])}")
print(f"  Binary/Boolean:        {len(binary_features)}")
print(f"  One-Hot Encoded:       {len(one_hot_features)}")

print(f"\nFeature Breakdown:")
print(f"  Original features:     {X.shape[1]}")
print(f"  Categorical encoded:   {len(encode_cats)} → {len(one_hot_features)} dummies")
print(f"  Net change:            +{X_encoded.shape[1] - X.shape[1]} features")

print("\n" + "="*80)

# Show sample features from each type
print("\n📌 Sample Features:")

# Sample numeric (non-binary)
sample_numeric = [f for f in numeric_features if f not in binary_features][:5]
if sample_numeric:
    print(f"\nNumeric (first 5): {', '.join(sample_numeric)}")

# Sample binary
sample_binary = binary_features[:5]
if sample_binary:
    print(f"\nBinary (first 5): {', '.join(sample_binary)}")

# Sample one-hot
sample_onehot = one_hot_features[:5]
if sample_onehot:
    print(f"\nOne-Hot Encoded (first 5): {', '.join(sample_onehot)}")

print("\n" + "="*80)


📋 FINAL FEATURE MATRIX COMPOSITION

Total Features:          179
  Numeric (continuous):  29
  Binary/Boolean:        150
  One-Hot Encoded:       133

Feature Breakdown:
  Original features:     55
  Categorical encoded:   9 → 133 dummies
  Net change:            +124 features


📌 Sample Features:

Numeric (first 5): flow, temperature, dissolved_o2, ph, conductivity

Binary (first 5): fecal_coliform_is_bdl, total_coliform_is_bdl, fecal_streptococci_is_bdl, total_kjeldahl_n_is_bdl, nitrate_n_is_bdl

One-Hot Encoded (first 5): type_water_body_ Dam, type_water_body_Creek, type_water_body_Dam, type_water_body_Lake, type_water_body_Nala



### Assemble Feature Matrix (No Scaling)

In [11]:
# ============================================================================
# FINAL FEATURE MATRIX
# ============================================================================

X_features = X_encoded.copy()

# Invariants
assert X_features.isna().sum().sum() == 0, "NaNs present after encoding"
assert X_features.shape[0] == y.shape[0], "Row mismatch between X and y"

print("✓ Feature matrix assembled and validated")


✓ Feature matrix assembled and validated


### Export Model-Ready Features

In [12]:
# ============================================================================
# EXPORT FEATURES (MODEL-READY, UN-SCALED)
# ============================================================================

OUT_CSV = os.path.join(DATA_DIR, "processed", "csv", "nwmp_features_v1.csv")
OUT_PARQUET = os.path.join(DATA_DIR, "processed", "parquet", "nwmp_features_v1.parquet")

X_features.assign(**{TARGET_COL: y}).to_csv(OUT_CSV, index=False)

try:
    X_features.assign(**{TARGET_COL: y}).to_parquet(OUT_PARQUET, index=False)
except Exception as e:
    print("Parquet export skipped:", e)

print(f"✓ Features exported to {OUT_CSV}")


✓ Features exported to /Users/rex/Documents/personal/AquaSafe/data/processed/csv/nwmp_features_v1.csv


### 🔍 Output Contract Validation

**Objective:** Verify that exported dataset meets quality standards and is ready for modeling

In [13]:
# ============================================================================
# OUTPUT CONTRACT VALIDATION
# ============================================================================
print("\n🔍 VALIDATING OUTPUT CONTRACT...")
print("="*80)

validation_passed = True

# Reload the exported file to verify
try:
    df_verify = pd.read_csv(OUT_CSV)
    print(f"  ✓ File successfully exported and reloaded: {OUT_CSV}")
except Exception as e:
    print(f"  ❌ FAILED: Could not reload exported file")
    print(f"     Error: {e}")
    validation_passed = False

if validation_passed:
    # Check 1: Shape preserved
    expected_rows = df.shape[0]
    expected_cols = X_encoded.shape[1] + 1  # +1 for target
    
    if df_verify.shape[0] == expected_rows:
        print(f"  ✓ Row count preserved: {df_verify.shape[0]}")
    else:
        print(f"  ❌ FAILED: Row count mismatch")
        print(f"     Expected: {expected_rows}, Got: {df_verify.shape[0]}")
        validation_passed = False
    
    if df_verify.shape[1] == expected_cols:
        print(f"  ✓ Column count correct: {df_verify.shape[1]} (features + target)")
    else:
        print(f"  ❌ FAILED: Column count mismatch")
        print(f"     Expected: {expected_cols}, Got: {df_verify.shape[1]}")
        validation_passed = False
    
    # Check 2: Target preserved
    if TARGET_COL in df_verify.columns:
        print(f"  ✓ Target column '{TARGET_COL}' present")
        
        if df_verify[TARGET_COL].nunique() == y.nunique():
            print(f"  ✓ Target classes preserved: {df_verify[TARGET_COL].nunique()} classes")
        else:
            print(f"  ❌ FAILED: Target classes lost")
            print(f"     Expected: {y.nunique()}, Got: {df_verify[TARGET_COL].nunique()}")
            validation_passed = False
    else:
        print(f"  ❌ FAILED: Target column missing from export")
        validation_passed = False
    
    # Check 3: No NaNs introduced
    nan_count = df_verify.isna().sum().sum()
    if nan_count == 0:
        print("  ✓ No missing values in exported file")
    else:
        print(f"  ❌ FAILED: {nan_count} NaN values found in exported file")
        validation_passed = False
    
    # Check 4: Data types preserved
    numeric_count = df_verify.select_dtypes(include=[np.number]).shape[1]
    if numeric_count > 0:
        print(f"  ✓ Numeric columns preserved: {numeric_count}")
    else:
        print(f"  ❌ FAILED: No numeric columns in export")
        validation_passed = False
    
    # Check 5: No duplicates introduced
    dup_count = df_verify.duplicated().sum()
    if dup_count == 0:
        print("  ✓ No duplicate rows")
    else:
        print(f"  ⚠️ WARNING: {dup_count} duplicate rows found")

print("="*80)

if validation_passed:
    print("\n✅ ALL OUTPUT CONTRACT VALIDATIONS PASSED!")
    print(f"✅ Model-ready dataset exported: {OUT_CSV}")
    print(f"   Records: {df_verify.shape[0]}")
    print(f"   Features: {df_verify.shape[1] - 1} (+ 1 target)")
    print(f"   Size: {os.path.getsize(OUT_CSV) / 1024:.2f} KB")
else:
    print("\n❌ OUTPUT CONTRACT VALIDATION FAILED!")
    print("   Please review the export process")

print("="*80)


🔍 VALIDATING OUTPUT CONTRACT...
  ✓ File successfully exported and reloaded: /Users/rex/Documents/personal/AquaSafe/data/processed/csv/nwmp_features_v1.csv
  ✓ Row count preserved: 171
  ✓ Column count correct: 180 (features + target)
  ✓ Target column 'use_based_class' present
  ✓ Target classes preserved: 4 classes
  ✓ No missing values in exported file
  ✓ Numeric columns preserved: 29
  ✓ No duplicate rows

✅ ALL OUTPUT CONTRACT VALIDATIONS PASSED!
✅ Model-ready dataset exported: /Users/rex/Documents/personal/AquaSafe/data/processed/csv/nwmp_features_v1.csv
   Records: 171
   Features: 179 (+ 1 target)
   Size: 175.84 KB


### 📝 Feature Registry

**Objective:** Create a machine-readable manifest of all features for deployment and documentation

In [14]:
# ============================================================================
# FEATURE REGISTRY (FOR DOCUMENTATION & DEPLOYMENT)
# ============================================================================
import json
from datetime import datetime

print("\n📝 Creating Feature Registry...")

# Create feature registry
feature_registry = {
    "metadata": {
        "version": "1.0",
        "created_at": datetime.now().isoformat(),
        "created_by": "feature_engineering_pipeline",
        "input_file": INPUT_CSV,
        "output_file": OUT_CSV
    },
    "dataset_info": {
        "total_records": len(df),
        "total_features": X_encoded.shape[1],
        "target_name": TARGET_COL,
        "target_classes": sorted(y.unique().tolist())
    },
    "feature_details": {
        "feature_names": X_encoded.columns.tolist(),
        "numeric_features": len([f for f in numeric_features if f not in binary_features]),
        "binary_features": len(binary_features),
        "one_hot_encoded_features": len(one_hot_features)
    },
    "encoding_strategy": {
        "method": "one_hot",
        "drop_first": False,
        "categorical_columns_encoded": encode_cats,
        "dummy_columns_created": len(one_hot_features)
    },
    "data_quality": {
        "missing_values": 0,
        "duplicate_rows": 0,
        "data_types": {
            "numeric": len(numeric_features),
            "categorical": len(encode_cats)
        }
    }
}

# Save feature registry
registry_path = os.path.join(DATA_DIR, "processed", "feature_registry_v1.json")
with open(registry_path, 'w') as f:
    json.dump(feature_registry, f, indent=2)

print(f"✓ Feature registry saved: {registry_path}")

# Print summary
print("\n📋 Feature Registry Summary:")
print("="*80)
print(f"Version:           {feature_registry['metadata']['version']}")
print(f"Total Features:    {feature_registry['dataset_info']['total_features']}")
print(f"Target:            {feature_registry['dataset_info']['target_name']}")
print(f"Classes:           {', '.join(feature_registry['dataset_info']['target_classes'])}")
print(f"Encoding Method:   {feature_registry['encoding_strategy']['method']}")
print(f"Registry Location: {registry_path}")
print("="*80)

# Optionally display first 20 feature names
print(f"\n📌 First 20 Features:")
for i, feat in enumerate(X_encoded.columns[:20], 1):
    print(f"   {i:2d}. {feat}")
if len(X_encoded.columns) > 20:
    print(f"   ... and {len(X_encoded.columns) - 20} more")

print("\n✅ Feature engineering complete and documented!")


📝 Creating Feature Registry...
✓ Feature registry saved: /Users/rex/Documents/personal/AquaSafe/data/processed/feature_registry_v1.json

📋 Feature Registry Summary:
Version:           1.0
Total Features:    179
Target:            use_based_class
Classes:           A, B, C, E
Encoding Method:   one_hot
Registry Location: /Users/rex/Documents/personal/AquaSafe/data/processed/feature_registry_v1.json

📌 First 20 Features:
    1. flow
    2. temperature
    3. dissolved_o2
    4. ph
    5. conductivity
    6. bod
    7. nitrate_n
    8. fecal_coliform
    9. total_coliform
   10. fecal_streptococci
   11. turbidity
   12. phenophelene_alkanity
   13. total_alkalinity
   14. chlorides
   15. cod
   16. total_kjeldahl_n
   17. amonia_n
   18. hardness_caco3
   19. calcium_caco3
   20. magnesium_caco3
   ... and 159 more

✅ Feature engineering complete and documented!
